In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arashomranpour","key":"efe346db89a76cced36a45b4b56d13ab"}'}

In [3]:
! mkdir ~/.kaggle


In [4]:

! cp kaggle.json ~/.kaggle/


In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets list


ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                        Student Performance Factors                          94KB  2024-09-02 10:53:57          25526        453  1.0              
ironwolf404/electric-vehicle-population-in-usa                 Electric Vehicle Population in USA                    6MB  2024-10-05 12:06:02            695         25  1.0              
abdulszz/spotify-most-streamed-songs                           Spotify Most Streamed Songs                          60KB  2024-09-07 18:23:14           9282        122  1.0              
valakhorasani/mobile-device-usage-and-user-behavior-dataset    Mo

In [7]:
!kaggle competitions download -c sentiment-analysis-on-movie-reviews

100% 1.90M/1.90M [00:01<00:00, 1.57MB/s]
100% 1.90M/1.90M [00:01<00:00, 1.38MB/s]


In [8]:
!ls

kaggle.json  sample_data  sentiment-analysis-on-movie-reviews.zip


In [9]:
!unzip sentiment-analysis-on-movie-reviews.zip

Archive:  sentiment-analysis-on-movie-reviews.zip
  inflating: sampleSubmission.csv    
  inflating: test.tsv.zip            
  inflating: train.tsv.zip           


In [10]:
!unzip test.tsv.zip

Archive:  test.tsv.zip
  inflating: test.tsv                


In [11]:
!unzip train.tsv.zip

Archive:  train.tsv.zip
  inflating: train.tsv               


In [12]:
!pip install pandas numpy transformers tensorflow tdqm

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1321 sha256=6bcadaaaa18026480ccea9ee38ba4e59bc18ddeeaba8e3bdaaa388a01f763176
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm.auto import tqdm
from transformers import BertTokenizer

In [15]:
df=pd.read_csv("./data/train.tsv",sep="\t")
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [17]:
df.drop("PhraseId",axis=1,inplace=True)

In [18]:
df["Sentiment"].value_counts()

,count
Sentiment,
2,79582
3,32927
1,27273
4,9206
0,7072


* 0 - Neg
* 1 - SNeg
* 2 - Neutral
* 3 - SPos
* 4 - Pos

In [19]:
tokenizer=BertTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
te=df["Phrase"].iloc[0]

In [21]:
tok=tokenizer.encode_plus(te,max_length=256,truncation=True,padding="max_length",add_special_tokens=True,return_tensors="tf")

In [ ]:
tok

In [22]:
X_input_id=np.zeros((len(df),256))
X_attention_mask=np.zeros((len(df),256))
X_attention_mask.shape

(156060, 256)

In [23]:
def gen(df,ids,masks,tokenizer):
  for i ,text in tqdm(enumerate(df["Phrase"])):
    tokenized=tokenizer.encode_plus(text,max_length=256,truncation=True,padding="max_length",add_special_tokens=True,return_tensors="tf")
    ids[i,:]=tokenized.input_ids
    masks[i,:]=tokenized.attention_mask

  return ids,masks

In [24]:
X_input_id,X_attention_mask=gen(df,X_input_id,X_attention_mask,tokenizer=tokenizer)

0it [00:00, ?it/s]

In [25]:
X_input_id

array([[  101.,   138.,  1326., ...,     0.,     0.,     0.],
       [  101.,   138.,  1326., ...,     0.,     0.,     0.],
       [  101.,   138.,  1326., ...,     0.,     0.,     0.],
       ...,
       [  101.,   170., 25247., ...,     0.,     0.,     0.],
       [  101.,   170., 25247., ...,     0.,     0.,     0.],
       [  101., 22572., 12148., ...,     0.,     0.,     0.]])

In [26]:
labels=np.zeros((len(df),5))

In [27]:
labels[np.arange(len(df)),df["Sentiment"].values]=1

In [ ]:
labels

In [63]:
dataset=tf.data.Dataset.from_tensor_slices((X_input_id,X_attention_mask,labels))

labelize the dataset

In [64]:
def sentimentmap(input_ids,attention_mask,labels):
  return{
      "input_ids":input_ids,
      "attention_mask":attention_mask,

  },labels



In [65]:
dataset=dataset.map(sentimentmap)


In [66]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [67]:
dataset=dataset.shuffle(10000).batch(16,drop_remainder=True)

In [68]:
a=0.8
train_size=int((len(df)//16)*a)
train_size

7802

In [69]:
train_df=dataset.take(train_size)
val_df=dataset.skip(train_size)

In [34]:
from transformers import TFBertModel
bert=TFBertModel.from_pretrained("bert-base-cased")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [70]:
input_ids=tf.keras.layers.Input(shape=(256,),name="input_ids",dtype='int32')
attention_masks=tf.keras.layers.Input(shape=(256,),name="attention_mask",dtype='int32')


bert_embd=bert.bert(input_ids,attention_mask=attention_masks)[1]
layer2=tf.keras.layers.Dense(512,activation="relu")(bert_embd)
output_layer=tf.keras.layers.Dense(5,activation="softmax")(layer2)


model=tf.keras.Model(inputs=[input_ids,attention_masks],outputs=output_layer)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 256, 7                                     

In [71]:
optim=tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func=tf.keras.losses.CategoricalCrossentropy()
acc=tf.keras.metrics.CategoricalAccuracy("accuracy")

In [72]:
model.compile(optimizer=optim,loss=loss_func,metrics=[acc])


In [73]:
train_df

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [74]:
hist=model.fit(train_df,validation_data=val_df,epochs=1)

7802/7802 [==============================] - 6824s 870ms/step - loss: 0.7856 - accuracy: 0.6724 - val_loss: 0.7798 - val_accuracy: 0.6736


In [75]:
model.save("sentiment_model")

In [79]:
load=tf.keras.models.load_model("sentiment_model")

In [91]:
text="movie was bad , i hate it"
def prepare_data(inpute,tokenizer):

  tokenized=tokenizer.encode_plus(text,max_length=256,truncation=True,padding="max_length",add_special_tokens=True,return_tensors="tf")

  return {
      "input_ids": tf.cast(tokenized.input_ids,tf.float64),
      "attention_mask": tf.cast(tokenized.attention_mask,tf.float64)
  }
tok=prepare_data(text,tokenizer)
p=load.predict(tok)
classes=["Negative","a little negative", "neutral","a little good","good"]
out=np.argmax(p[0])
print(classes[out])

1/1 [==============================] - 0s 64ms/step
Negative


In [92]:
text="movie was great"
tok=prepare_data(text,tokenizer)
p=load.predict(tok)
out=np.argmax(p[0])
print(classes[out])

1/1 [==============================] - 0s 67ms/step
good
